In [13]:
import os
from sys import path
dir_actual = os.getcwd()
path.insert(0, 'C:/Users/guill/OneDrive/Documentos/Tesis/Proyect')

import myfuncions.randomize_f as myrand
import myfuncions.core_shell_f as cs
import myfuncions.tools_f as tools
import subprocess as sbp
import myfuncions.graphs_f as gf
from myfuncions.lammpin_f import wlammpin
import time
import PDF.runner as pdf

coords = cs.cubeShell(14,2.775,'fcc')
atpos, eleList = cs.atpos_eleList_maker(coords,'Pt')
atpos, eleList = tools.spherical_cut(atpos, eleList, 25)
atpos, eleList, mode = myrand.cambiar_elementos_prob(atpos,{'Pt':0.28, 'Pd':0.45, 'Co':0.27})
tools.show_percent(atpos, eleList)

Corta esfericamente al atpos
Cambia elementos prob a atpos
atomos totales : 4321
Co : 26.868780374913214 %, 1161
Pd : 45.31358481832909 %, 1958
Pt : 27.817634806757695 %, 1202


In [14]:
temps = [300, 400, 500, 600]

for i in temps:
    name = f'p{i}.xyz'
    tools.show_percent(atpos, eleList)

    try: 
        os.remove(name)
        os.remove(name[:-4]+'.ini')
        os.remove(name[:-4]+'_d.xyz')
        os.remove(name[:-4]+'_d2.xyz')
        os.remove(name[:-4]+'_d2.txt')
    except: pass

    tools.wXYZ(atpos, name)
    wlammpin(atpos,eleList,name)


    tf = i

    input = f"""
    # 2NN EAM potential in LAMMPS

     units		metal
     atom_style	atomic
     boundary	f f f

     read_data	{name[:-4]}.ini
     pair_style	meam
     pair_coeff	* * ..\\..\\LAMMPS\\library-PtPdCo.meam Pt Pd Co ..\\..\\LAMMPS\\PtPdCo.pot Pt Pd Co
  
     neighbor	0.5 bin
     neigh_modify	every 1 delay 0 check yes once no
     velocity	all create 0.00 13478 rot yes mom yes dist gaussian
     timestep	0.005

     fix            1 all nve
     fix            2 all temp/rescale 10 0.00 0.00 1.0 1
     thermo         100
     thermo_style   custom step temp atoms pe ke etotal
     thermo_modify  lost ignore
     run 600

     unfix 2
     fix         3 all temp/berendsen 0.0 {tf} 0.1
     thermo		100
     run		3000

     unfix 3
     timestep   0.001
     thermo		100
     fix		4 all langevin {tf} {tf} 0.5 87654
     dump       4 all xyz 1 dump_temp.xyz
     undump     4
     run        1
     dump       4 all xyz 599 {name[:-4]}_d.xyz
     run		599
     """
    with open('input.in','w') as archivo:
        archivo.write(input)

    os.system('start powershell -Command lmp -in input.in; Start-Sleep -s 10; exit')
    #os.system('start powershell -Command lmp -in input.in')
    time.sleep(200)
    print(f'MD for {i} done!!')

    datpos, deleList = tools.leer_xyz(f'{name[:-4]}_d.xyz')
    datpos, deleList = tools.cambiar_etiquetas_xyz(datpos, deleList, {1: 'Pt', 2: 'Pd', 3:'Co'})
    tools.show_percent(datpos, ['Pt','Pd','Co'])
    tools.wXYZ(datpos,f'{name[:-4]}_d2.xyz')

    
    pdf.run_pdf(f'{name[:-4]}_d2.xyz', dir_actual, 2600,0.02)



atomos totales : 4321
Co : 26.868780374913214 %, 1161
Pd : 45.31358481832909 %, 1958
Pt : 27.817634806757695 %, 1202
p300.ini
MD for 300 done!!
Cambia etiquetas a "atpos"
jala
atomos totales : 4321
Pt : 27.817634806757695 %, 1202
Pd : 45.31358481832909 %, 1958
Co : 26.868780374913214 %, 1161
c:\Users\guill\OneDrive\Documentos\Tesis\Proyect\26_de_abril\Pruebas_temperatura

 -0.160999998      -7.52890015      -22.7029991              27   215171.000    
  -3.95870996      -3.68446994      -22.7203999              46   215171.000    
  -3.87100019E-02  -3.72238994      -22.6980000              78   215171.000    
        4321  atomos      9333360  pares 
 Rango de distancias    2.23643231     -   49.0219116    
 Distancias promedio    24.9493923    
 Factor de normalizacion    458840.938    

atomos totales : 4321
Co : 26.868780374913214 %, 1161
Pd : 45.31358481832909 %, 1958
Pt : 27.817634806757695 %, 1202
p400.ini
MD for 400 done!!
Cambia etiquetas a "atpos"
jala
atomos totales : 4321
P

In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
M = pd.DataFrame()

df_exp = pd.read_csv('../Post-PtPdCo-PDF.csv', sep = '\s+', header=None, names=['r','G(r)'])
df_exp = df_exp.replace('--',np.nan).dropna()
rest_df_exp = df_exp["G(r)"].iloc[1::2].astype(float)
rest_df_exp = rest_df_exp.reset_index(drop=True)
rest_df_exp.index += 1
M['exp'] = rest_df_exp

dx = 0.02

#M.index = M0['#r(A)']
M300 = pd.read_csv('p300_d2.txt', sep='\s+')
M400 = pd.read_csv('p400_d2.txt', sep='\s+')
M500 = pd.read_csv('p500_d2.txt', sep='\s+')
M600 = pd.read_csv('p600_d2.txt', sep='\s+')
M['300 K'] = M300['G(r)']
M['400 K'] = M400['G(r)']
M['500 K'] = M500['G(r)']
M['600 K'] = M600['G(r)']
M.index = (M.index + 1)*dx

for i in temps:
    a = M[M[f'{i} K']== max(M[f'{i} K'])].index[0]
    b = 0.29 #### Al parecer este es el originario de todo lo que yo estaba haciendo
    c = b/(float(M[f'{i} K'][a]))
    M[f'{i} K'] *= c
    rest = M['exp'] - M[f'{i} K']
    error = np.sum(rest**2)
    error_p = np.sqrt(error)*100/50
    print(f'Para {i} K el error es {error},y el error porcentual es {error_p}')




fig = px.line(M, template='simple_white', title='Pair Distribution Functions of 9 Models',
    width=1000, height=600)
fig.update_layout(
    font_family="serif",
    title_font_family="serif")
fig.update_xaxes(title_text='G(r)')
fig.update_yaxes(title_text='r [A]')
fig.show()

Para 300 K el error es 2.6918916336957954,y el error porcentual es 3.2813970401009356
Para 400 K el error es 2.224728151677601,y el error porcentual es 2.98310452493881
Para 500 K el error es 1.9790095714850426,y el error porcentual es 2.8135455009542976
Para 600 K el error es 1.5975087034296038,y el error porcentual es 2.527851817990607
